## Setup Comprehend Through CLI/API:
https://docs.aws.amazon.com/comprehend/latest/dg/get-started-customclass.html

## Setup Comprehend Through AWS Console 

https://docs.aws.amazon.com/comprehend/latest/dg/getting-started-document-classification.html

Good example of using Comprehend for Positive/Negative Sentiment:  https://github.com/aws-samples/amazon-comprehend-custom-entity/blob/master/3-AWS-Comprehend-Negative-Custom-Classifier.ipynb

## Make sure this SageMakerNotebookExecutionRole has access to Comprehend

In [2]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [3]:
comprehend = boto3.client('comprehend')

In [4]:
%store -r noheader_train_s3_uri

print(noheader_train_s3_uri)

s3://sagemaker-us-east-2-533787958253/data/amazon_reviews_us_Digital_Software_v1_00_noheader.csv


In [5]:
!aws s3 ls $noheader_train_s3_uri

2020-04-29 14:19:18   15164581 amazon_reviews_us_Digital_Software_v1_00_noheader.csv


## Create Data Access Role for Comprehend

## Create Policy

In [6]:
assume_role_policy_doc = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "comprehend.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
} 

## Create Role and Attach Policies

In [7]:
#iam_comprehend_role_name = 'DSOAWS_Comprehend'
iam_comprehend_role_name = 'COEAWS_Comprehend'

In [8]:
import json
import boto3
from botocore.exceptions import ClientError

try:
    iam = boto3.client('iam')

    iam_role_comprehend = iam.create_role(
        RoleName=iam_comprehend_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description='COEAWS Comprehend Role'
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        iam_role_comprehend = iam.get_role(RoleName=iam_comprehend_role_name)
        print("Role already exists")
    else:
        print("Unexpected error: %s" % e)

If you have and Unexpected error above you need to create an IAM inline access policy and inline attach

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "jsr1",
            "Effect": "Allow",
            "Action": [
                "iam:CreateRole"
            ],
            "Resource": [
                "*"
            ]
        }
    ]
}
```

In [9]:
comprehend_s3_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "s3:GetObject"
            ],
            "Resource": [
                "arn:aws:s3:::{}/*".format(bucket)
            ],
            "Effect": "Allow"
        },
        {
            "Action": [
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket)
            ],
            "Effect": "Allow"
        },
        {
            "Action": [
                "s3:PutObject"
            ],
            "Resource": [
                "arn:aws:s3:::{}/*".format(bucket)
            ],
            "Effect": "Allow"
        }
    ]
}

print(comprehend_s3_policy_doc)


{'Version': '2012-10-17', 'Statement': [{'Action': ['s3:GetObject'], 'Resource': ['arn:aws:s3:::sagemaker-us-east-2-533787958253/*'], 'Effect': 'Allow'}, {'Action': ['s3:ListBucket'], 'Resource': ['arn:aws:s3:::sagemaker-us-east-2-533787958253'], 'Effect': 'Allow'}, {'Action': ['s3:PutObject'], 'Resource': ['arn:aws:s3:::sagemaker-us-east-2-533787958253/*'], 'Effect': 'Allow'}]}


In [10]:
response = iam.put_role_policy(
    RoleName=iam_comprehend_role_name,
    PolicyName='COEAWS_ComprehendPolicyToS3',
    PolicyDocument=json.dumps(comprehend_s3_policy_doc)
)

If you have an error you'll need to add PutRolePolicy

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "jsr2",
            "Effect": "Allow",
            "Action": [
                "iam:PutRolePolicy"
            ],
            "Resource": [
                "*"
            ]
        }
    ]
}
```

## Train the Model

In [11]:
prefix = 'models'

s3_output_job = 's3://{}/{}/{}'.format(bucket, prefix, 'comprehend/output')
print(s3_output_job)

s3://sagemaker-us-east-2-533787958253/models/comprehend/output


In [12]:
iam_role_comprehend_arn = iam_role_comprehend['Role']['Arn']

In [13]:
import datetime

id = str(datetime.datetime.now().strftime("%s"))

training_job = comprehend.create_document_classifier(
    DocumentClassifierName='Amazon-Customer-Reviews-Classifier-'+ id,
    DataAccessRoleArn=iam_role_comprehend_arn,
    InputDataConfig={
        'S3Uri': noheader_train_s3_uri
    },
    OutputDataConfig={
        'S3Uri': s3_output_job
    },
    LanguageCode='en'
)

If the above has errors you'll need to create a PassRole policy

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "jsr3",
            "Effect": "Allow",
            "Action": [
                "iam:PassRole"
            ],
            "Resource": [
                "*"
            ]
        }
    ]
}
```

In [14]:
import time

jobArn = training_job['DocumentClassifierArn']

max_time = time.time() + 3 * 60 * 60 # 3 hours
while time.time() < max_time:
    describe_custom_classifier = comprehend.describe_document_classifier(
        DocumentClassifierArn = jobArn
    )
    status = describe_custom_classifier["DocumentClassifierProperties"]["Status"]
    print("Custom classifier: {}".format(status))
    
    if status == "TRAINED" or status == "IN_ERROR":
        print('')
        print('Status {}'.format(status))
        print('')
        print(describe_custom_classifier["DocumentClassifierProperties"])
        break
        
    time.sleep(5)

Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAI

Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: T

In [46]:
!aws s3 ls 

2020-03-04 22:00:41 coe-ai-video
2020-03-04 19:39:23 coe-cal-ai-polly-anil
2020-03-04 17:56:16 coe-faas
2020-03-26 17:28:18 coe-opm-faces
2020-03-09 16:58:45 rekognition-video-console-demo-cmh-533787958253-tubndls8jejq0n
2020-03-03 18:41:08 sagemaker-studio-0qnahwofwbs
2020-03-03 15:56:05 sagemaker-studio-3rk9wz4j1uh
2020-03-03 18:32:07 sagemaker-studio-fbc9rb6owm
2020-03-03 17:45:10 sagemaker-studio-k06y0dzfxg
2020-03-03 18:51:13 sagemaker-studio-k0n6398mj8
2020-03-03 17:35:55 sagemaker-studio-mwp36qinptr
2020-03-03 18:41:30 sagemaker-studio-qlfop46x0ji
2020-03-03 17:48:24 sagemaker-studio-wa13v0wmlme
2020-04-27 14:05:08 sagemaker-us-east-2-533787958253


## Show Results of the Classifier

In [47]:
print(describe_custom_classifier["DocumentClassifierProperties"])

{'DocumentClassifierArn': 'arn:aws:comprehend:us-east-2:533787958253:document-classifier/Amazon-Customer-Reviews-Classifier-1588350035', 'LanguageCode': 'en', 'Status': 'TRAINED', 'SubmitTime': datetime.datetime(2020, 5, 1, 16, 20, 35, 361000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2020, 5, 1, 16, 59, 40, 62000, tzinfo=tzlocal()), 'TrainingStartTime': datetime.datetime(2020, 5, 1, 16, 22, 50, 848000, tzinfo=tzlocal()), 'TrainingEndTime': datetime.datetime(2020, 5, 1, 16, 57, 28, 837000, tzinfo=tzlocal()), 'InputDataConfig': {'S3Uri': 's3://sagemaker-us-east-2-533787958253/data/amazon_reviews_us_Digital_Software_v1_00_noheader.csv'}, 'OutputDataConfig': {'S3Uri': 's3://sagemaker-us-east-2-533787958253/models/comprehend/output/533787958253-CLR-c38b32ce24d337245e9009550ff06cae/output/output.tar.gz'}, 'ClassifierMetadata': {'NumberOfLabels': 5, 'NumberOfTrainedDocuments': 31005, 'NumberOfTestDocuments': 3445, 'EvaluationMetrics': {'Accuracy': 0.5202, 'Precision': 0.5304, 'Recall':

In [48]:
model_arn = describe_custom_classifier["DocumentClassifierProperties"]["DocumentClassifierArn"]
print(model_arn)

arn:aws:comprehend:us-east-2:533787958253:document-classifier/Amazon-Customer-Reviews-Classifier-1588350035


In [49]:
import os
#Retrieve the S3URI from the model output and create jobkey variable.
job_output = describe_custom_classifier["DocumentClassifierProperties"]["OutputDataConfig"]["S3Uri"]
print(job_output)

path_prefix = 's3://{}/'.format(bucket)

job_key = os.path.relpath(job_output, path_prefix)

print(job_key)

s3://sagemaker-us-east-2-533787958253/models/comprehend/output/533787958253-CLR-c38b32ce24d337245e9009550ff06cae/output/output.tar.gz
models/comprehend/output/533787958253-CLR-c38b32ce24d337245e9009550ff06cae/output/output.tar.gz


In [50]:
!aws s3 cp $job_output ./output.tar.gz

download: s3://sagemaker-us-east-2-533787958253/models/comprehend/output/533787958253-CLR-c38b32ce24d337245e9009550ff06cae/output/output.tar.gz to ./output.tar.gz


In [51]:
#Download the model metrics
s3 = boto3.resource('s3')

s3.Bucket(bucket).download_file(job_key, './output.tar.gz')

In [52]:
#Unpack the gzip file
!tar xvzf ./output.tar.gz

output/
output/confusion_matrix.json


In [53]:
import json

with open('./output/confusion_matrix.json') as json_file:
    data = json.load(json_file)
print(json.dumps(data, indent=2, default=str))

{
  "confusion_matrix": [
    [
      411,
      208,
      50,
      11,
      9
    ],
    [
      125,
      340,
      177,
      34,
      13
    ],
    [
      56,
      182,
      312,
      108,
      31
    ],
    [
      13,
      42,
      163,
      320,
      151
    ],
    [
      16,
      20,
      43,
      201,
      409
    ]
  ],
  "labels": [
    "1",
    "2",
    "3",
    "4",
    "5"
  ],
  "type": "multi_class",
  "all_labels": [
    "1",
    "2",
    "3",
    "4",
    "5"
  ]
}


In [55]:
!pip install tabulate

You are using pip version 10.0.1, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [56]:
from IPython.display import HTML, display
import tabulate
table = [['', '1', '2', '3', '4', '5', '(Predicted)'],
         ['1', data['confusion_matrix'][0][0], data['confusion_matrix'][0][1], data['confusion_matrix'][0][2], data['confusion_matrix'][0][3], data['confusion_matrix'][0][4]],
         ['2', data['confusion_matrix'][1][0], data['confusion_matrix'][1][1], data['confusion_matrix'][1][2], data['confusion_matrix'][1][3], data['confusion_matrix'][1][4]],
         ['3', data['confusion_matrix'][2][0], data['confusion_matrix'][2][1], data['confusion_matrix'][2][2], data['confusion_matrix'][2][3], data['confusion_matrix'][2][4]],
         ['4', data['confusion_matrix'][3][0], data['confusion_matrix'][3][1], data['confusion_matrix'][3][2], data['confusion_matrix'][3][3], data['confusion_matrix'][3][4]],
         ['5', data['confusion_matrix'][4][0], data['confusion_matrix'][4][1], data['confusion_matrix'][4][2], data['confusion_matrix'][4][3], data['confusion_matrix'][4][4]],
         ['(Actual)']]
display(HTML(tabulate.tabulate(table, tablefmt='html')))

,1,2,3,4,5,(Predicted)
1,411,208,50,11,9,
2,125,340,177,34,13,
3,56,182,312,108,31,
4,13,42,163,320,151,
5,16,20,43,201,409,
(Actual),,,,,,


## Deploy Endpoint

In [57]:
inference_endpoint_response = comprehend.create_endpoint(
    EndpointName='comprehend-inference-endpoint',
    ModelArn = model_arn,
    DesiredInferenceUnits = 1
)

In [58]:
endpoint_arn = inference_endpoint_response["EndpointArn"]
print(endpoint_arn)

arn:aws:comprehend:us-east-2:533787958253:document-classifier-endpoint/comprehend-inference-endpoint


## Predict with Endpoint

In [59]:
describe_response = comprehend.describe_endpoint(
    EndpointArn = endpoint_arn
)
print(describe_response)

{'EndpointProperties': {'EndpointArn': 'arn:aws:comprehend:us-east-2:533787958253:document-classifier-endpoint/comprehend-inference-endpoint', 'Status': 'CREATING', 'ModelArn': 'arn:aws:comprehend:us-east-2:533787958253:document-classifier/Amazon-Customer-Reviews-Classifier-1588350035', 'DesiredInferenceUnits': 1, 'CurrentInferenceUnits': 0, 'CreationTime': datetime.datetime(2020, 5, 1, 18, 12, 35, 261000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 5, 1, 18, 12, 35, 261000, tzinfo=tzlocal())}, 'ResponseMetadata': {'RequestId': '4fd7303b-ead1-453e-add7-1ed80852ad9f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4fd7303b-ead1-453e-add7-1ed80852ad9f', 'content-type': 'application/x-amz-json-1.1', 'content-length': '402', 'date': 'Fri, 01 May 2020 18:13:04 GMT'}, 'RetryAttempts': 0}}


In [60]:
import time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_response = comprehend.describe_endpoint(
        EndpointArn = endpoint_arn
    )
    status = describe_response["EndpointProperties"]["Status"]
    print("Endpoint: {}".format(status))
    
    if status == "IN_SERVICE" or status == "IN_ERROR":
        break
        
    time.sleep(5)

Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CREATING
Endpoint: CR

In [61]:
txt = """I loved it!  I will recommend this to everyone."""

response = comprehend.classify_document(
    Text= txt,
    EndpointArn = endpoint_arn
)

import json
print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "5",
      "Score": 0.9981356263160706
    },
    {
      "Name": "4",
      "Score": 0.0018844201695173979
    },
    {
      "Name": "3",
      "Score": 1.0016706255555619e-05
    }
  ],
  "ResponseMetadata": {
    "RequestId": "edd5d8c7-9f33-4b32-9d68-d19d2339dca3",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "edd5d8c7-9f33-4b32-9d68-d19d2339dca3",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "139",
      "date": "Fri, 01 May 2020 18:25:11 GMT"
    },
    "RetryAttempts": 0
  }
}


In [62]:
txt = """Really bad.  I hope they don't make this anymore."""

response = comprehend.classify_document(
    Text= txt,
    EndpointArn = endpoint_arn
)

import json
print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "2",
      "Score": 0.8435407876968384
    },
    {
      "Name": "3",
      "Score": 0.08270315825939178
    },
    {
      "Name": "1",
      "Score": 0.07377045601606369
    }
  ],
  "ResponseMetadata": {
    "RequestId": "51740b3b-09bd-4bbc-a0c1-0c024780133f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "51740b3b-09bd-4bbc-a0c1-0c024780133f",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "135",
      "date": "Fri, 01 May 2020 18:25:42 GMT"
    },
    "RetryAttempts": 0
  }
}


In [63]:
txt = """It's OK."""

response = comprehend.classify_document(
    Text= txt,
    EndpointArn = endpoint_arn
)

import json
print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "3",
      "Score": 0.9962722659111023
    },
    {
      "Name": "4",
      "Score": 0.0035591258201748133
    },
    {
      "Name": "5",
      "Score": 0.00018858080147765577
    }
  ],
  "ResponseMetadata": {
    "RequestId": "d642110d-d8d9-41b1-9978-bff8cf20477e",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "d642110d-d8d9-41b1-9978-bff8cf20477e",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "139",
      "date": "Fri, 01 May 2020 18:28:57 GMT"
    },
    "RetryAttempts": 0
  }
}


In [64]:
pprint(json.dumps(response, indent=2, default=str))

Pretty printing has been turned OFF
